# Load Model from Huggingface Hub

To use a model with the Transformer Extension one needs to download it from the Huggingface Hub and put it into the BucketFS. Please refer to the Transformer Extension <a href="https://github.com/exasol/transformers-extension/blob/main/doc/user_guide/user_guide.md" target="_blank" rel="noopener">User Guide</a> to find more information about model loading functionality it provides.

There are two ways of doing this.

1. Using the TE_MODEL_DOWNLOADER_UDF UDF.
2. Downloading a model to a local drive and subsequently uploading it into the BucketFS using CLI or an API.

The first method requires the database machine to have internet access. The second method provides a workaround if this is a problem. Another advantage of the second method is that it caches downloaded models on the local drive. This can make the model transfer quicker if it needs to be repeated.

<b>This notebook is not supposed to be run on its own. It contains model loading functions that are called by other notebooks.</b>

## Loading model with UDF

Here is the first way of loading the model. We wrap it into a function so that other notebooks can call it.

In [ ]:
def load_huggingface_model_udf(model_name: str, conf: "Secrets") -> None:
    """
    Loads specified model into BucketFS using a UDF provided with the Transformer Extension.

    model_name     - Name of the model at Huggingface hub, e.g. facebook/nllb-moe-54b.
    conf           - Sandbox configuration object.
    """

    import pyexasol

    sql = f"""
    SELECT {sb_config.SCHEMA}.TE_MODEL_DOWNLOADER_UDF(
        '{model_name}',
        '{sb_config.TE_MODELS_BFS_DIR}',
        '{sb_config.TE_BFS_CONN}',
        '{sb_config.TE_TOKEN_CONN}'
    )
    """

    dsn = f'{sb_config.EXTERNAL_HOST_NAME}:{sb_config.HOST_PORT}'
    with pyexasol.connect(dsn=dsn, user=sb_config.USER, password=sb_config.PASSWORD, compression=True) as conn:
        conn.execute(query=sql)

## Loading model using the notebook

Here is the second way of loading the model. This method is very similar to the command line interface. For details on how to use the CLI please refer to the Transformer Extension <a href="https://github.com/exasol/transformers-extension/blob/main/doc/user_guide/user_guide.md" target="_blank" rel="noopener">User Guide</a>.

In [ ]:
def load_huggingface_model_cli(model_name: str, conf: "Secrets", force_download: bool = False) -> None:
    """
    Loads specified model into BucketFS by saving it first to a local drive, as per the command-line interface.

    model_name     - Name of the model at Huggingface hub, e.g. facebook/nllb-moe-54b.
    conf           - Sandbox configuration object.
    force_download - If True the model will be reloaded from the hub even if it has been cached before.
    """

    from pathlib import Path
    import re
    from transformers import AutoTokenizer, AutoModel
    from exasol_transformers_extension.utils.bucketfs_operations import (
        create_bucketfs_location, get_model_path, upload_model_files_to_bucketfs)

    # Make a name for the model sub-directory
    sub_dir = re.sub(r"[/\\?%*:|\"<>\x7F\x00-\x1F]", "-", model_name)
    model_dir = str(Path(conf.get('TE_MODELS_CACHE_DIR')) / sub_dir)
    
    # Maybe download the model from the Huggingface hub
    AutoTokenizer.from_pretrained(model_name, cache_dir=model_dir, token=sb_config.TE_TOKEN, force_download=force_download)
    AutoModel.from_pretrained(model_name, cache_dir=model_dir, token=sb_config.TE_TOKEN, force_download=force_download)

    # Create bucketfs location
    bucketfs_location = create_bucketfs_location(
        conf.get('BUCKETFS_SERVICE'), conf.get('EXTERNAL_HOST_NAME'), int(conf.get('BUCKETFS_PORT')),
        conf.get('BUCKETFS_USE_HTTPS').lower() == 'true', conf.get('BUCKETFS_USER'), conf.get('BUCKETFS_PASSWORD'), 
        conf.get('BUCKETFS_BUCKET'), conf.get('TE_BFS_DIR'))

    # Upload the downloaded model files into bucketfs
    upload_path = get_model_path(conf.get('TE_MODELS_BFS_DIR'), model_name)
    upload_model_files_to_bucketfs(model_dir, upload_path, bucketfs_location)

## Method selector

This is the main entry point. The call will be dispatched to one of the above functions depending on the selected method.

In [ ]:
def load_huggingface_model(model_name: str, conf: "Secrets", method: str = None, force_download: bool = False) -> None:
    """
    Loads specified model into BucketFS choosing one of the two available methods.

    model_name     - Name of the model at Huggingface hub, e.g. facebook/nllb-moe-54b.
    conf           - Sandbox configuration object.
    method         - The recognized values are "udf" and "cli". If the parameter is not set then
                     will look for method selection in the configuration. That failed the CLI method
                     is used. 
    force_download - If True the model will be reloaded from the hub even if it has been cached before.
    """

    if not method:
        method = conf.get('TE_LOAD_METHOD')
    if method:
        method = method.lower()
    if method == 'udf':
        load_huggingface_model_udf(model_name, conf)
    else:
        load_huggingface_model_cli(model_name, conf, force_download=force_download)